<a href="https://colab.research.google.com/github/Abdansb/NoCopyZone/blob/main/link_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install bs4
! pip install selenium
! apt-get update
! apt install chromium-chromedriver
! pip install pyvirtualdisplay
! apt-get install xvfb
! pip install ipython-autotime
%load_ext autotime

import re
import html
import json
import time
import requests
import random
import urllib3
import logging
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
from pyvirtualdisplay import Display
from selenium import webdriver
from selenium.webdriver.common.by import By

# Load link dataset

In [2]:
#import columns A and C from Excel file
df = pd.read_excel('/content/Copy of 231219_Digital_Content_Protection_Vidio_Appendix.xlsx',
                   sheet_name='data',
                   usecols='F',
                   index_col = None )
print(df[:10])

                                         Content URL
0  https://www.facebook.com/61552914750614/videos...
1  https://www.facebook.com/61552361257367/videos...
2  https://www.facebook.com/100065268481709/video...
3  https://www.facebook.com/61551633585049/videos...
4  https://www.facebook.com/61553458406539/videos...
5  https://www.facebook.com/61552300508309/videos...
6  https://www.facebook.com/61552917830854/videos...
7  https://www.facebook.com/61552938333393/videos...
8  https://www.facebook.com/61552725683391/videos...
9  https://www.facebook.com/61553042637294/videos...
time: 21.3 s (started: 2023-12-25 18:35:40 +00:00)


# Main

In [17]:
user_agents = [
	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
  'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
  'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'
]
user_agent = random.choice(user_agents)

display = Display(visible=0, size=(320, 240)).start()  # visible=0
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-plugins-discovery");
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--no-sandbox");
chrome_options.add_argument("--disable-dev-shm-usage");
chrome_options.add_argument('--disable-popup-blocking')
chrome_options.add_argument(f"user-agent={user_agent}")

def string_check(link):
  keywords_to_search = [
      "missing",
      "removed",
      "doesn't exist",
      "deleted",
      "not found",
      "forbidden",
      "blocked",
      "page not found",
      "404 error",
      "content removed",
      "no longer available",
      "this page isn't available",]
  user_agent = random.choice(user_agents)
  chrome_options.add_argument(f"user-agent={user_agent}")
  driver = webdriver.Chrome(options=chrome_options)
  driver.delete_all_cookies()
  driver.set_window_size(800,800)
  driver.set_window_position(0,0)

  driver.get(link)
  time.sleep(2)
  html = driver.page_source
  #print(html)
  driver.save_screenshot('screenie.png')

  if any(keyword.lower() in html.lower() for keyword in keywords_to_search):
      result = "Removed"
  else:
      result = "Active"

  return result


#url_to_check = "https://tv1.bioskop21official.com/nonton-eps/pertaruhan-the-series-season-1-episode-1/"
#string_check(url_to_check)



time: 90.5 ms (started: 2023-12-25 18:51:30 +00:00)


In [7]:
import csv

def append_to_csv(input_string):
    # Split the input string into parts
    parts = input_string.split(', ')

    # Extract values
    source = parts[0]
    url = parts[1]
    status = parts[2]
    # Open the CSV file in append mode
    with open("Output.csv", mode='a', newline='', encoding='utf-8') as csv_file:
        # Create a CSV writer object
        csv_writer = csv.writer(csv_file)
        # Append the values to the CSV file
        csv_writer.writerow([source, url, status])

# Example usage
#input_string = "Content URL, https://www.facebook.com/100080210871960/videos/2088931548125338, Removed"

# Call the function to append the input string to the CSV file
#append_to_csv(input_string)


time: 997 µs (started: 2023-12-25 18:37:29 +00:00)


In [20]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

ranged_df = df[99:150]
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}


def url_code_status(pandas_dataframe):
  for index, row in pandas_dataframe.iterrows():
    try:
      response = requests.get(url = row["Content URL"],
                              allow_redirects=False,
                              headers=headers,
                              verify=False)

      if response.status_code == 200:
        result = string_check(row["Content URL"])
        print(f"{list(df.columns)[0]}, {row['Content URL']}, {result}")
        append_to_csv(f"{list(df.columns)[0]}, {row['Content URL']}, {result}")
      else:
        print(f"{list(df.columns)[0]}, {row['Content URL']}, {response.status_code}")
        append_to_csv(f"{list(df.columns)[0]}, {row['Content URL']}, {response.status_code}")


    except Exception as e:
        #print(f'[ERROR]: {e}')
        print(f"{list(df.columns)[0]}, {row['Content URL']}, ERROR")
        append_to_csv(f"{list(df.columns)[0]}, {row['Content URL']}, ERROR")
        continue

url_code_status(ranged_df)



Content URL, https://tapenoads.com/v/YW1oLXlz4XSvboQ/NGEFILM21.Married.with.Senior.Ep.06.720p.mp4, 404
Content URL, https://hncrirpotl4l.lions-vidcdn.com/hls2/01/00845/cmcp6r94iqsh_,l,n,h,.urlset/index-f1-v1-a1.m3u8?t=NhGEfjUL9w1rfRAFJ4W4odHM2fI8AUaTJU02J3l_6SE&s=1701669501&e=129600&f=4229998&srv=rM7JzcUItghM&i=0.4&sp=1500&p1=rM7JzcUItghM&p2=rM7JzcUItghM&asn=24961, 403
Content URL, https://qcgdhivljuj6.lions-vidcdn.com/hls2/01/00845/9zi85oups8h5_,l,n,h,.urlset/index-f1-v1-a1.m3u8?t=Ry6pcDaqvsmRTWZbCy5r_l52u23ssvxRLBguo3fqqWE&s=1701669509&e=129600&f=4229999&srv=rM7JzcUItghM&i=0.4&sp=1500&p1=rM7JzcUItghM&p2=rM7JzcUItghM&asn=24961, 403
Content URL, https://98zd3iu9j1s8.lions-vidcdn.com/hls2/01/00846/o8yblk20j5ep_,l,n,h,.urlset/index-f1-v1-a1.m3u8?t=RqCL8lYd0HOUC-I9k4jvuBW_M9lHm-XOw8HXPT1SH3U&s=1701669524&e=129600&f=4230000&srv=rM7JzcUItghM&i=0.4&sp=1500&p1=rM7JzcUItghM&p2=rM7JzcUItghM&asn=24961, 403
Content URL, https://pjg3itk8bndw.lions-vidcdn.com/hls2/01/00846/5j7y8il9bgh8_,l,n,h,.urls

# Facebook

In [146]:
my_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
display = Display(visible=0, size=(320, 240)).start()  # visible=0
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-plugins-discovery");
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--no-sandbox");
chrome_options.add_argument("--disable-dev-shm-usage");
driver = webdriver.Chrome(options=chrome_options)
driver.delete_all_cookies()
driver.set_window_size(800,800)
driver.set_window_position(0,0)

main_page = 'https://tv1.bioskop21official.com/nonton-eps/pertaruhan-the-series-season-1-episode-1/'
def facebook_check(link):
  driver.get(link)
  time.sleep(2)
  html = driver.page_source
  driver.save_screenshot('screenie.png')
  keyword = "This Video Isn't Available Anymore"
  result = "Removed" if keyword in html else "Active"
  #print(result)
  return result

#facebook_check(main_page)


'Active'

time: 7.2 s (started: 2023-12-25 14:28:36 +00:00)


# Print page source

In [ ]:
import requests

r = requests.get("https://www.facebook.com/100080210871960/videos/2088931548125338",
                 allow_redirects=True,
                 headers=headers,
                 verify=False)
print(r.text)

# Selenium Testing

In [45]:

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import InvalidSessionIdException
from selenium.common.exceptions import InvalidArgumentException
from selenium.common.exceptions import ElementNotInteractableException


time: 1.37 ms (started: 2023-12-25 10:05:54 +00:00)


In [ ]:
my_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #headless means no graphical user interface
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable_gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument(f"--user-agent={my_user_agent}")
chrome_options.add_argument('--enable-javascript')

chrome_options.add_argument("start-maximized")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

driver = webdriver.Chrome(options=chrome_options)
time.sleep(5)
main_page = 'https://www.facebook.com/100080210871960/videos/2088931548125338'
driver.get(main_page)
html = driver.page_source

soup = BeautifulSoup(html)
print(soup)


# Test Google ISP

In [102]:
!curl -s https://packagecloud.io/install/repositories/ookla/speedtest-cli/script.deb.sh | sudo bash
!sudo apt-get install speedtest
! printf 'YES\n' | speedtest


You may only use this Speedtest software and information generated
from it for personal, non-commercial use, through a command line
interface on a personal computer. Your use of this software is subject
to the End User License Agreement, Terms of Use and Privacy Policy at
these URLs:

	https://www.speedtest.net/about/eula
	https://www.speedtest.net/about/terms
	https://www.speedtest.net/about/privacy


Do you accept the license? [type YES to accept]: License acceptance recorded. Continuing.


   Speedtest by Ookla

      Server: Farmers Telephone Cooperative, Inc. - Sumter, SC (id: 6806)
         ISP: Google Cloud
Idle Latency:    19.97 ms   (jitter: 0.14ms, low: 19.75ms, high: 20.05ms)
    Download:  6002.29 Mbps (data used: 8.4 GB)                                                   
                 88.22 ms   (jitter: 23.35ms, low: 18.75ms, high: 182.10ms)
      Upload:   943.55 Mbps (data used: 1.1 GB)                                                   
                120.51 ms   (